In [1]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()


BACKEND_URL = os.getenv("BACKEND_URL") or "http://127.0.0.1:8000"
BACKEND_URL

'http://127.0.0.1:8000'

In [8]:
def play(card1, card2, position, argument_pour_inference):
    ''' Returns LLM's action given cards in hands and position at the table (UTG or BB)'''
    if position == 'UTG' :
        data = {   
        "variant" : "NT",
        "game_id" : 779459871,
        "hand_nb" : 0,
        "small_blind" : 0.25,
        "big_blind" : 0.50,
        "min_bet" : 0.25,
 
        "players" : ["n0hvn", "tbmfc", "naprimer", "Log_in", "IlxxxlI", "gmjohn", "MANTISGUYV10", "BiGFck"],
        "starting_stacks" : [55.50, 28.47, 55.31, 15.15, 20, 28.76, 57.49, 17],
        "players_seats" : [1, 2, 3, 4, 5, 7, 8, 9],
 
        "button_seat" : 2,
        "player_small_blind" : "naprimer",
        "player_big_blind" : "Log_in",
 
        "player" : "IlxxxlI",
        "cards_player" : [card1, card2],
 
        "dealed_cards" : {
                    "flop": [],
                    "turn": [],
                    "river": []
                   },
 
 
        "actions" : {"pre_flop" : {"players": [],
                            "actions": [],
                            "value": []},
                "post_flop" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_turn" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_river" : {
                            "players": [],
                            "actions": [],
                            "value": []
                                }
               },
 
 
        "winners" : [],
        "finishing_stacks": [],
        "card_shown_by_players" : []
        }

    elif position == 'BB':
        data = {   "variant" : "NT",
        "game_id" : 779460276,
        "hand_nb" : 0,
 
        "small_blind" : 0.25,
        "big_blind" : 0.50,
        "min_bet" : 0.25,
 
        "players" : ["n0hvn", "tbmfc", "naprimer", "Log_in", "IlxxxlI", "gmjohn", "MANTISGUYV10", "BiGFck"],
        "starting_stacks" : [55.50, 28.47, 55.06, 14.65, 20, 28.76, 58.24, 17],
        "players_seats" : [1, 2, 3, 4, 5, 7, 8, 9],
 
        "button_seat" : 3,
        "player_small_blind" : "Log_in",
        "player_big_blind" : "IlxxxlI",
 
        "player" : "IlxxxlI",
        "cards_player" : [card1, card2],
 
        "dealed_cards" : {
                    "flop": [],
                    "turn": [],
                    "river": []
                   },
 
 
        "actions" : {"pre_flop" : {"players": ["gmjohn", "MANTISGUYV10", "BiGFck", "n0hvn", "tbmfc", "naprimer", "Log_in"],
                            "actions": ["f", "cc", "f", "f", "f", "f", "f"],
                            "value": [None, None, None, None, None, None, None]},
                "post_flop" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_turn" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_river" : {
                            "players": [],
                            "actions": [],
                            "value": []
                                }
               },
 "response"
 
        "winners" : [],
        "finishing_stacks": [],
        "card_shown_by_players" : []
        }
    
    #Envoie l'inférence dans le back  action = inference(data, argument_pour_inference)
    #Envoie le json data et reçoit l'action
    
    route = "/predict_test"
    route = "/predict"
    response = requests.post(f'{BACKEND_URL}{route}', json=data)
    response_json = response.json()
    if 'response' in response_json:
        action = response_json['response']
        return action
    else:
        raise ValueError(f"Error in response: {response_json}")


In [9]:
card1 = "2s"
card2 = "2h"
position = "UTG"
argument_pour_inference = "blabla"
action = play(card1, card2, position, argument_pour_inference)
action

ValueError: Error in response: {'detail': 'Status: 401. Message: {\n  "message":"Unauthorized",\n  "request_id":"3dad4e7ed0f805790d4bcc1d6bc06726"\n}'}

In [12]:
import numpy as np
import random

def range_hands():
    '''Generate hands for all possible pairs (suited and off-suited)'''
    range_matrix = np.zeros((13, 13), dtype=object)
    suits = ["s", "h", "d", "c"]
    values = ["A", "K", "Q", "J", "10", "9", "8", "7", "6", "5", "4", "3", "2"]
    
    for i in range(13):
        for j in range(13):
            if j > i:  # Suited hands
                color = random.choice(suits)
                card1, card2 = values[i] + color, values[j] + color
                range_matrix[i][j] = card1, card2
            else:  # Off-suited hands
                colors = random.sample(suits, 2)
                card1, card2 = values[i] + colors[0], values[j] + colors[1]
                range_matrix[i][j] = card1, card2

    return range_matrix

# Example
print(range_hands())


[[('Ad', 'Ah') ('Ad', 'Kd') ('Ad', 'Qd') ('Ah', 'Jh') ('Ah', '10h')
  ('As', '9s') ('Ah', '8h') ('As', '7s') ('Ac', '6c') ('As', '5s')
  ('Ah', '4h') ('Ac', '3c') ('Ad', '2d')]
 [('Kh', 'As') ('Kh', 'Kd') ('Ks', 'Qs') ('Kh', 'Jh') ('Kc', '10c')
  ('Kh', '9h') ('Kc', '8c') ('Kh', '7h') ('Ks', '6s') ('Kc', '5c')
  ('Kh', '4h') ('Ks', '3s') ('Kc', '2c')]
 [('Qs', 'Ah') ('Qd', 'Ks') ('Qd', 'Qc') ('Qd', 'Jd') ('Qh', '10h')
  ('Qc', '9c') ('Qs', '8s') ('Qd', '7d') ('Qs', '6s') ('Qh', '5h')
  ('Qs', '4s') ('Qc', '3c') ('Qc', '2c')]
 [('Jd', 'Ah') ('Js', 'Kh') ('Js', 'Qd') ('Jc', 'Jd') ('Jh', '10h')
  ('Jh', '9h') ('Js', '8s') ('Jh', '7h') ('Jd', '6d') ('Jd', '5d')
  ('Jh', '4h') ('Jd', '3d') ('Js', '2s')]
 [('10d', 'As') ('10s', 'Kc') ('10s', 'Qc') ('10s', 'Jc') ('10c', '10s')
  ('10c', '9c') ('10d', '8d') ('10h', '7h') ('10c', '6c') ('10h', '5h')
  ('10h', '4h') ('10c', '3c') ('10s', '2s')]
 [('9d', 'Ah') ('9s', 'Kc') ('9c', 'Qh') ('9h', 'Jd') ('9c', '10d')
  ('9d', '9h') ('9d', '8d') ('9c',

In [ ]:
def model_range_generator(position, argument_pour_inference):
    """Fulfill a range with LLM's actions"""
    range_matrix = range_hands()
    for i in range(13):
        for j in range(13):
            card1, card2 = range_matrix[i][j]
            action = play(card1, card2, position, argument_pour_inference)
            range_matrix[i][j] = action

    return range_matrix

In [ ]:
def range_accuracy(range1, range2):
    '''Calcule accuracy of range1 according to a reference range range2'''
    matching_actions = 0
    for i in range(13):
        for j in range(13):
            if range1[i][j] == range2[i][j] :
                matching_actions += 1
    return matching_actions/(13*13)